In [1]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

url = 'https://www.stproperty.sg'
driver=webdriver.Chrome()           
driver.get("https://www.stproperty.sg/property-for-rent/max-rent-price-50000/max-floor-10000/min-bedroom-0/max-bedroom-10/max-psf-5000/from-sg/page1/size-10/sort-date-desc")    
time.sleep(5)                         

urllist = []

data = driver.page_source
soup = BeautifulSoup(data, 'lxml')
jobs = soup.findAll('a',{'class':'list-container-pic'})   
for job in jobs:
    urllist.append(url+job['href'])  
for i in range(1,160): # 1600 items
    driver.find_element_by_link_text('Next').click()
    time.sleep(5)
    data = driver.page_source
    soup = BeautifulSoup(data, 'lxml')
    jobs = soup.findAll('a',{'class':'list-container-pic'})     
    for job in jobs:
        urllist.append(url+job['href'])


In [2]:
import xlwt
f = xlwt.Workbook()
sheet1 = f.add_sheet(u'sheet1',cell_overwrite_ok=True)
for i in range(len(urllist)):
    sheet1.write(i,0,urllist[i])
f.save('url.xls')

In [3]:
import xlrd
workbook=xlrd.open_workbook("url.xls") 
worksheet=workbook.sheet_by_index(0)
urllist=worksheet.col_values(0)
print(urllist[1000])

https://www.stproperty.sg/condo-for-rent/condominium-for-rent/pavilion-11/37511152


In [4]:
import xlwt
import re
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

datalist = [['agent','title','street_name','post_code','area','bed_num','bath_num','posting_date','description','property_name','property_type',
            'price','price_psf','tenure','built_in_area','detail','furnished','floor_location','built_year','lease_term','mrt_1','mrt_1_distance',
            'mrt_2','mrt_2_distance','mrt_3','mrt_3_distance','school_1','school_1_distance','school_2','school_2_distance','school_3',
            'school_3_distance','supermarket_1','supermarket_1_distance','supermarket_2','supermarket_2_distance','supermarket_3',
            'supermarket_3_distance','childcare_1','childcare_1_distance','childcare_2','childcare_2_distance','childcare_3',
            'childcare_3_distance','facilities_1','facilities_2','unit_information','lat','lng']]

def getData(soup,static_soup):
    agent = soup.select('div.col-xs-12.agent-section div.row div.col-xs-12 div.col-xs-7 strong')[0].string if soup.select('div.col-xs-12.agent-section div.row div.col-xs-12 div.col-xs-7 strong') else ""
    title = soup.find('h4',{'class':'title-detail-page'}).string.strip() if soup.find('h4',{'class':'title-detail-page'}) else ""
    street_name = soup.find('a',{'title':'Street Name'}).string if soup.find('a',{'title':'Street Name'}) else ""
    post_code = str(soup.find('div',{'class':'page-header'}).contents[4].contents[2]).replace(',','').replace('<br/>','')
    area = soup.find('div',{'class':'page-header'}).contents[4].contents[5].string
    bed_num = soup.find('span',{'class':'icon-bed'}).string if soup.find('span',{'class':'icon-bed'}) else ""
    bath_num = soup.find('span',{'class':'icon-bathroom'}).string if soup.find('span',{'class':'icon-bathroom'}) else ""
    posting_date = str(soup.select('div.box-col640.pull-right.default-box-detail div.row div.col-xs-6 div.row div.col-xs-7')[-1])[22:-6]
    description = str(soup.select('div.detail-tab-content p')[1]).replace('<p>','').replace('<br/>','').replace('</p>','')
    property_name = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/search/')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/search/')}) else ""
    property_type = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/\w+-for-rent/[\w-]+-for-rent$')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/\w+-for-rent/[\w-]+-for-rent$')}) else ""
    price = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/\w+-for-rent/min-rent-price')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/\w+-for-rent/min-rent-price')}) else ""
    price_psf = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/\w+-for-rent/min-psf')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/\w+-for-rent/min-psf')}) else ""
    tenure = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/\w+-for-rent/[\w-]+-for-rent/')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/\w+-for-rent/[\w-]+-for-rent/')}) else ""
    built_in_area = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/\w+-for-rent/min-floor-')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-rent/condo-for-rent/min-floor-')}) else ""
    
    detail = ''
    detail_tag = soup.select('div.detail-tab-content div.row div.col-xs-6 div.row div.col-xs-6 span')
    for index in range(len(detail_tag)):
        detail = detail + detail_tag[index].string.strip() + '###'
    furnished = detail_tag[-4].string.strip()
    floor_location = detail_tag[-3].string.strip()
    built_year = detail_tag[-2].string.strip()
    lease_term = detail_tag[-1].string.strip()
    
    mrt_1 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')}) else ""
    mrt_1_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')})[0].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')}) else ""
    mrt_2 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')})[1].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')}) else ""
    mrt_2_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')})[1].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')}) else ""
    mrt_3 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')})[2].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')}) else ""
    mrt_3_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')})[2].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/nearby-mrt-')}) else ""
    school_1 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')}) else ""
    school_1_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')})[0].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')}) else ""
    school_2 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')})[1].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')}) else ""
    school_2_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')})[1].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')}) else ""
    school_3 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')})[2].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')}) else ""
    school_3_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')})[2].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-school-')}) else ""
    supermarket_1 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')}) else ""
    supermarket_1_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')})[0].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')}) else ""
    supermarket_2 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')})[1].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')}) else ""
    supermarket_2_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')})[1].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')}) else ""
    supermarket_3 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')})[2].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')}) else ""
    supermarket_3_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')})[2].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-mall-')}) else ""
    childcare_1 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')})[0].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')}) else ""
    childcare_1_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')})[0].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')}) else ""
    childcare_2 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')})[1].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')}) else ""
    childcare_2_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')})[1].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')}) else ""
    childcare_3 = soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')})[2].string if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')}) else ""
    childcare_3_distance = str(soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')})[2].next_sibling)[2:-1] if soup.findAll(name='a',attrs={"href":re.compile(r'^/property-for-sale/near-childcare-')}) else ""
    facilities_1 = str(soup.select('div.detail-tab-content div.row div.col-xs-12 div.row')[0]).replace('<div class="row">','').replace('<div class="col-xs-3">',',').replace('</div>','').strip() if soup.select('div.detail-tab-content div.row div.col-xs-12 div.row') else ""
    facilities_2 = str(soup.find('div',{'id':'facilities'}).contents[1]).replace('<div class="detail-tab-content">','').replace('<p>','').replace('</p>','').replace('<br/>','').replace('</div>','').replace('<hr/>','').replace('<strong>','').replace('</strong>',':').strip() if soup.find('div',{'id':'facilities'}) else ""
    unit_information = str(soup.find('div',{'id':'unit-information'}).contents[1].contents[4]).strip() if soup.find('div',{'id':'unit-information'}) else ""

    lat = ''
    lng = ''
    if static_soup.findAll(name='script',attrs={"src":re.compile(r'https://maps.googleapis.com/')}):
        tag = static_soup.findAll(name='script',attrs={"src":re.compile(r'https://maps.googleapis.com/')})[0].next_siblings
        coordinate = []
        for item in tag:
            coordinate.append(item)
        gmap = str(coordinate[1])
        lat = re.findall(r'var lat=(.*);',gmap)[0]
        lng = re.findall(r'var lng=(.*);',gmap)[0]

    return [agent,title,street_name,post_code,area,bed_num,bath_num,posting_date,description,property_name,property_type,
            price,price_psf,tenure,built_in_area,detail,furnished,floor_location,built_year,lease_term,mrt_1,mrt_1_distance,
            mrt_2,mrt_2_distance,mrt_3,mrt_3_distance,school_1,school_1_distance,school_2,school_2_distance,school_3,
            school_3_distance,supermarket_1,supermarket_1_distance,supermarket_2,supermarket_2_distance,supermarket_3,
            supermarket_3_distance,childcare_1,childcare_1_distance,childcare_2,childcare_2_distance,childcare_3,
            childcare_3_distance,facilities_1,facilities_2,unit_information,lat,lng]


driver=webdriver.Chrome() 
for url in urllist:
    tempurl = re.split(r'/(\d+)$',url)
    static_url = tempurl[0]+'/view-location/'+tempurl[1]
    response  = requests.get(static_url)
    static_soup = BeautifulSoup(response.content, 'lxml')
    driver.get(url)
    time.sleep(4)
    data = driver.page_source
    soup = BeautifulSoup(data, 'lxml')
    row = getData(soup,static_soup)
    datalist.append(row)


In [5]:
import xlwt
f = xlwt.Workbook()
sheet1 = f.add_sheet(u'sheet1',cell_overwrite_ok=True)
for i in range(len(datalist)):
    for j in range(len(datalist[0])):
        data = str(datalist[i][j])
        sheet1.write(i,j,data)
f.save('data.xls')

"\nimport xlwt\nf = xlwt.Workbook()\nsheet1 = f.add_sheet(u'sheet1',cell_overwrite_ok=True)\nfor i in range(len(datalist)):\n    for j in range(len(datalist[0])):\n        data = str(datalist[i][j])\n        sheet1.write(i,j,data)\nf.save('data1.xls')\n"